# Segmenting and Clustering Neighborhoods in Toronto

In [147]:
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

### Get the neighborhood dataframe

In [1]:
# Get the data from wiki
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Parse the html data using BeautifulSoup
soup = BeautifulSoup(wiki_url, 'html.parser')

# Find table in the website containing the data we need
postal_table = soup.find('table', attrs = {'rules': 'all'})

# Create a dataframe
column = ['PostalCode', 'Borough', 'Neighborhood']
df_toronto = pd.DataFrame(columns = column)

# Filter every td and insert to df_toronto

for td in postal_table.find_all('td'):
    if td.span.text != 'Not assigned':
        postal_code = td.find('b').text
        borough = td.find('span').text.split('(')[0]
        neighbor = td.find('span').text.split('(')[1].strip(')').replace(' /', ',').replace(')', ' ').strip(' ')
    
        df_toronto = df_toronto.append({'PostalCode': postal_code, 'Borough': borough, 'Neighborhood': neighbor}, ignore_index = True)
        
# Replace the borough names which is too long
df_toronto['Borough']=df_toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

### Get the coordinate data from the csv provided in the material.

In [26]:
# Get the coordinate data from the csv
import wget
loc_url = 'https://cocl.us/Geospatial_data'
loc_file = wget.download(loc_url)
df_loc = pd.read_csv(loc_file)
df_loc

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### Merge the 2 dataframe.

In [42]:
# Change the column name of df_loc
df_loc.columns = ['PostalCode', 'Latitude', 'Longitude']

# Merge the 2 dataframe
toronto_neighbor = pd.merge(df_toronto, df_loc[['PostalCode', 'Latitude', 'Longitude']], on = 'PostalCode')
toronto_neighbor

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### Explore how many borough and neighborhoods we get in the dataset.

In [43]:
print('The dataframe has {} boroughs and {} neighborhoods'.format(len(toronto_neighbor['Borough'].unique()),
                                                                  toronto_neighbor.shape[0]))

The dataframe has 15 boroughs and 103 neighborhoods


### Use geopy library to get the latitude and longitude values of Toronto

In [47]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = 'tor_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are ({}, {})'.format(latitude, longitude))

The geographical coordinate of Toronto are (43.6534817, -79.3839347)


### Create a map of Toronto with neighborhoods on top.

In [53]:
# Create map using coordinates of Toronto
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# Add markers on the map
for lat, lng, borough, neighborhood in zip(toronto_neighbor['Latitude'], toronto_neighbor['Longitude'], toronto_neighbor['Borough'], toronto_neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lng], radius = 5, popup = label, color = 'blue', fill = True, fill_color = '#3186cc', fill_opacity = 0.7, parse_html = False).add_to(map_toronto)
    
map_toronto

### Define Foursquare Credentials and Version

In [54]:
Client_ID = 'RJXDLADYUPSUOVK2FCAJAOKZCRZBOMK4F2CIRBJ30OIXJRLK'
Client_Secret = 'GLUI5A5ZIGN0410G2NFPVF0VNEWDDSX4DTR15QGMDFJIP24T'
Version = '20190425'

### Create a function to explore neighborhoods in Toronto

In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            Client_ID,
            Client_Secret,
            Version,
            lat,
            lng,
            radius,
            Limit)
        
        # Make a GET request
        result = requests.get(url).json()['response']['groups'][0]['items']
        
        # Return only relevant information for each nearby venue
        venues_list.append([(name,
                             lat,
                             lng,
                             v['venue']['name'],
                             v['venue']['location']['lat'],
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in result])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return (nearby_venues)

In [82]:
Limit = 100
toronto_venues = getNearbyVenues(names = toronto_neighbor['Neighborhood'],
                                 latitudes = toronto_neighbor['Latitude'],
                                 longitudes = toronto_neighbor['Longitude'])

In [103]:
print(toronto_venues.shape)
toronto_venues.head()

(2138, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


### Analyze Each Neighborhood

In [120]:
# Onehot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = '', prefix_sep = '')

# Add neighborhood column back to dataframe
toronto_onehot['Neighborhood_Name'] = toronto_venues['Neighborhood']

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head(10)

(2138, 279)


,Neighborhood_Name,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group rows by neighborhood and take the mean of frequency of occurrence of each category

In [127]:
toronto_grouped = toronto_onehot.groupby('Neighborhood_Name').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(100, 279)


,Neighborhood_Name,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Print each neighborhood along with the top 5 most common venues

In [128]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood_Name']:
    print('----' +hood+ '----')
    temp = toronto_grouped[toronto_grouped['Neighborhood_Name'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink   0.2
1             Breakfast Spot   0.2
2                     Lounge   0.2
3  Latin American Restaurant   0.2
4             Clothing Store   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1             Gym  0.12
2    Skating Rink  0.12
3  Sandwich Place  0.12
4     Coffee Shop  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.08
1                Coffee Shop  0.08
2                  Gift Shop  0.04
3                Bridal Shop  0.04
4  Middle Eastern Restaurant  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4               Museum  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.09
1           Sandw

### Put the result into a dataframe

In [129]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0: num_top_venues]

In [154]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# Create a new dataframe
toronto_neighbor_sorted = pd.DataFrame(columns = columns)
toronto_neighbor_sorted['Neighborhood'] = toronto_grouped['Neighborhood_Name']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_neighbor_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
toronto_neighbor_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Lounge,Latin American Restaurant,Clothing Store,Accessories Store,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Sandwich Place,Coffee Shop,Pub,Pharmacy,Organic Grocery,Optical Shop,Massage Studio
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Gift Shop,Bridal Shop,Middle Eastern Restaurant,Shopping Mall,Mobile Phone Shop,Fried Chicken Joint,Sandwich Place,Supermarket
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Museum,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Comfort Food Restaurant,Indian Restaurant,Pub,Restaurant,Café,Butcher,Fast Food Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,Pizza Place,Coffee Shop,Discount Store,Pharmacy,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Movie Theater
96,"Willowdale, Newtonbrook",Park,Accessories Store,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
97,Woburn,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
98,Woodbine Heights,Skating Rink,Beer Store,Park,Athletics & Sports,Video Store,Curling Ice,Bus Stop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [165]:
# Set number of clusters
k = 5

toronto_cluster = toronto_grouped.drop('Neighborhood_Name', axis = 1)

# Run k-means clustering
kmeans = KMeans(n_clusters = k, random_state = 0).fit(toronto_cluster)

Create a new dataframe that include the cluster and the top 10 venues for each neighborhood

In [174]:
# Add cluster labels
#toronto_neighbor_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_neighbor

# Merge the 2 dataframe
toronto_merged = toronto_merged.join(toronto_neighbor_sorted.set_index('Neighborhood'), on = 'Neighborhood')

# Drop the rows without cluster labels
toronto_merged = toronto_merged.dropna()
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Fast Food Restaurant,Food & Drink Shop,Park,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Financial or Legal Service,Hockey Arena,Portuguese Restaurant,Coffee Shop,Mediterranean Restaurant,Medical Center,Massage Studio,Men's Store,Metro Station,Movie Theater
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Bakery,Café,Park,Pub,Breakfast Spot,Theater,Sushi Restaurant,Bank,Electronics Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Carpet Store,Shoe Store,Miscellaneous Shop,Event Space,Boutique,Vietnamese Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,3,Coffee Shop,Sushi Restaurant,Yoga Studio,Japanese Restaurant,Mexican Restaurant,Café,Fried Chicken Joint,Sandwich Place,Beer Bar,Nightclub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,3,Pool,River,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,3,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Pub,Fast Food Restaurant,Men's Store,Mediterranean Restaurant,Yoga Studio
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558,3,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Park,Recording Studio,Restaurant,Skate Park,Farmers Market,Burrito Place
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,0,Baseball Field,Park,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant


Visualize the result

In [193]:
# Create map
tor_map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

# Set color scheme for clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
color_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in color_array]

# Add marker on the map
markers_color = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' ,Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker([lat, lon], radius = 5, popup = label, color = rainbow[cluster - 1], fill = True, fill_color = rainbow[cluster - 1], fill_opacity = 0.7).add_to(tor_map_clusters)
    
tor_map_clusters

### Examine Clusters

Cluster 0

In [189]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,Baseball Field,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant,Movie Theater
101,Etobicoke,Baseball Field,Park,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant


Cluster 1

In [188]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Fast Food Restaurant,Food & Drink Shop,Park,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
21,York,Park,Women's Store,Pool,Accessories Store,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
35,East York/East Toronto,Park,Intersection,Convenience Store,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
40,North York,Playground,Airport,Construction & Landscaping,Park,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
46,North York,Bank,Park,Grocery Store,Shopping Mall,Accessories Store,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
52,North York,Park,Accessories Store,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
61,Central Toronto,Park,Swim School,Bus Line,Accessories Store,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
64,York,Park,Accessories Store,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station
66,North York,Convenience Store,Park,Accessories Store,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
85,Scarborough,Playground,Intersection,Coffee Shop,Park,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop


Cluster 2

In [190]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,Bar,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant,Movie Theater


Cluster 3

In [191]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,Financial or Legal Service,Hockey Arena,Portuguese Restaurant,Coffee Shop,Mediterranean Restaurant,Medical Center,Massage Studio,Men's Store,Metro Station,Movie Theater
2,Downtown Toronto,Coffee Shop,Bakery,Café,Park,Pub,Breakfast Spot,Theater,Sushi Restaurant,Bank,Electronics Store
3,North York,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Carpet Store,Shoe Store,Miscellaneous Shop,Event Space,Boutique,Vietnamese Restaurant
4,Queen's Park,Coffee Shop,Sushi Restaurant,Yoga Studio,Japanese Restaurant,Mexican Restaurant,Café,Fried Chicken Joint,Sandwich Place,Beer Bar,Nightclub
6,Scarborough,Fast Food Restaurant,Accessories Store,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,Coffee Shop,Café,Gym,Hotel,Japanese Restaurant,Deli / Bodega,Sushi Restaurant,Restaurant,Asian Restaurant,Bakery
98,Etobicoke,Pool,River,Accessories Store,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant
99,Downtown Toronto,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Pub,Fast Food Restaurant,Men's Store,Mediterranean Restaurant,Yoga Studio
100,East Toronto Business,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Park,Recording Studio,Restaurant,Skate Park,Farmers Market,Burrito Place


cluster 4

In [192]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,Bakery,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant,Movie Theater
